In [ ]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set global variables
PROJECT = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks"
OUTPUTS = rf"{PROJECT}\Outputs"
DATA = rf"{PROJECT}\Data"
DATA_IN = rf"{DATA}\Data_in"
DATA_PROC = rf"{DATA}\Data_proc"
DATA_OUT = rf"{DATA}\Data_out"
ERA5_DATA = rf"Z:\WB Data\ERA5 Reanalysis\monthly"

## Variable definitions

- Drought: Monthly cumulative precipitation **2.5 standard deviations below** the monthly historical average, considering 30 years to estimate historical averages and SD
- Excessive Rain: Monthly cumulative precipitation **2.5 standard deviations above** the monthly historical average, considering 30 years to estimate historical averages and SD
- Heatwave: More than 6 days with temperature **over 40 degrees** and 2.5 standard deviations higher than monthly historical average, considering 30 years to estimate historical averages and SD. Furthermore, **warm spell** duration index is higher than 6 days.
- Coldwave: More than 6 days with temperature **below 0 degrees** and 2.5 standard deviations lower than monthly historical average, considering 30 years to estimate historical averages and SD. Furthermore, **cold spell** duration index is higher than 6 days.
  ​

In [ ]:
files = os.listdir(ERA5_DATA)
datasets = []
for file in tqdm(files):
    ds = xr.open_dataset(os.path.join(ERA5_DATA, file))
    datasets += [ds]
precipitation = xr.concat(datasets, dim="time")
precipitation.to_netcdf(os.path.join(DATA_OUT, "ERA5_monthly_1970-2021.nc"))

In [ ]:
# Longitude is in range 0-360, with 0 at Greenwich. We need to transform it to -180 to 180
def transform_longitude(longitude):
    if longitude > 180:
        return longitude - 360
    else:
        return longitude

precipitation['longitude'] = precipitation['longitude'].to_series().apply(transform_longitude).values
precipitation = precipitation.sortby('longitude').sortby("latitude")
precipitation = precipitation.rename({'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
precipitation

In [ ]:
import climate_indices

## Original paper: https://www.droughtmanagement.info/literature/AMS_Relationship_Drought_Frequency_Duration_Time_Scales_1993.pdf
## User guide to SPI: https://digitalcommons.unl.edu/cgi/viewcontent.cgi?article=1208&context=droughtfacpub
#   It is recommended to use SPI-9 or SPI-12 to compute droughts.
#   "SPI values below -1.5 for these timescales (SPI-9) are usually a good indication that dryness is having a significant impact on
#    agriculture and may be affecting other sectors as well."
## More here: https://www.researchgate.net/profile/Sorin-Cheval/publication/264467702_Spatiotemporal_variability_of_the_meteorological_drought_in_Romania_using_the_Standardized_Precipitation_Index_SPI/links/5842d18a08ae2d21756372f8/Spatiotemporal-variability-of-the-meteorological-drought-in-Romania-using-the-Standardized-Precipitation-Index-SPI.pdf
## Ignore negative values, they are normal: https://confluence.ecmwf.int/display/UDOC/Why+are+there+sometimes+small+negative+precipitation+accumulations+-+ecCodes+GRIB+FAQ

# Parameters
distribution = climate_indices.indices.Distribution.gamma
data_start_year = 1990
calibration_year_initial = 1990
calibration_year_final = 2020
periodicity = climate_indices.compute.Periodicity.monthly


da_precip_groupby = precipitation["tp"].stack(point=('lat', 'lon')).groupby('point')

# apply SPI to each `point`
spis = []
for i in [1, 3, 6, 9, 12]:
    print(f"Computing SPI-{i}")
    da_spi = xr.apply_ufunc(climate_indices.indices.spi,
                            da_precip_groupby,
                            i,
                            distribution,
                            data_start_year,
                            calibration_year_initial,
                            calibration_year_final,
                            periodicity)
    da_spi = da_spi.unstack('point').rename(f"spi{i}")
    da_spi.to_netcdf(os.path.join(DATA_OUT, f"ERA5_monthly_1970-2021_SPI{i}.nc"))
    spis += [da_spi]

In [ ]:
#### Construct indicators
climate_data = xr.combine_by_coords(spis + [precipitation["t2m"]])

## Export data
climate_data.to_netcdf(rf"{DATA_OUT}/Climate_shocks_v3_previous_months.nc")

In [ ]:
import pandas as pd
pd.read_csv(r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\births_climate_20000.csv")